# Map data visualizations
This visualizes some map data for any location.

In [ ]:
import numpy as np
import pandas as pd
import pyproj
import shapely
import osmnx
import pygeohash
import geopandas
import folium
import os
import copy

import plotly
from plotly.subplots import make_subplots
from plotly.graph_objects import Scatter

from geo_encodings import draw_shape
import mgm_utilities


## Parameters

In [ ]:
# aoi_lat, aoi_lon = 42.424200, -71.146093  # Mystic Lakes Arlington
aoi_lat, aoi_lon = 42.938609, -71.050340

# AOIs are squares of this dimension, in meters.
aoi_size = 10000


In [ ]:
def get_projections(lon, lat):

    center_lat = lat
    center_lon = lon
    x0 = aoi_size / 2
    y0 = aoi_size / 2
    
    proj_def = f"""
    +proj=tmerc +lat_0={center_lat} +lon_0={center_lon} 
    +k=1.0 +x_0={x0} +y_0={y0} +datum=WGS84 +units=m +no_defs
    """
    
    ltm_crs = pyproj.CRS.from_proj4(proj_def)
    wgs84_crs = pyproj.CRS.from_epsg(4326)
    proj_forward = pyproj.Transformer.from_crs(wgs84_crs, ltm_crs, always_xy=True).transform
    proj_inverse = pyproj.Transformer.from_crs(ltm_crs, wgs84_crs, always_xy=True).transform

    return proj_forward, proj_inverse

In [ ]:
# Define a polygon for the AOI bounds.

def get_aoi_bbox(aoi_size):
    x0, y0 = 0, 0
    x1, y1 = aoi_size, aoi_size
    xx = [x0, x1, x1, x0, x0]
    yy = [y0, y0, y1, y1, y0]
    aoi_bbox = shapely.Polygon(list(zip(xx, yy)))
    return aoi_bbox

In [ ]:
tiles_ll = []
tile_contents = []

# Get a string identifier for this AOI.
aoi_id = pygeohash.encode(latitude=aoi_lat, longitude=aoi_lon, precision=8)
print('aoi_id', aoi_id)

# Get forward and inverse projections.
proj_forward, proj_inverse = get_projections(aoi_lon, aoi_lat)

# Use that projection to define lon/lat bounds for the query below. Make sure the bounds go 
# a little farther out than necessary to avoid edge artifacts from map projections.
buf = 100
x0, y0 = 0, 0
x1, y1 = aoi_size, aoi_size
lon0, lat0 = proj_inverse(x0 - buf, y0 - buf)
lon1, lat1 = proj_inverse(x1 + buf, y1 + buf)
query_bounds = [lon0, lat0, lon1, lat1]

print('re-projected query bounds, with buffer:')
print(proj_forward(lon0, lat0))
print(proj_forward(lon1, lat1))

# Query for all the geospatial entities we need within the bounding box.
tags = {
    'landuse': True,
    'place': True,
    'highway': True,
    'railway': True,
    'aeroway': True,
    'bridge': True,
    'tunnel': True,
    'power': True,
    'natural': True,
    'water': True,
    'waterway': True,
    'landcover': True,
    'building': True,
    'amenity': True,
    'shop': True,
    'leisure': True
}
features = osmnx.features.features_from_bbox(query_bounds, tags=tags).reset_index()
print('%d features from OSM' % len(features))

# Re-format and filter everything.
# BTW, "gents" is "geospatial entities".
gents = [] 
for feature in features.to_dict('records'):
    
    geomxy = shapely.ops.transform(proj_forward, feature['geometry'])
    if geomxy.is_empty:
        continue
    gtype = geomxy.geom_type

    for rule in mgm_utilities.rules:
        if gtype == rule['gtype']:
            osm_key = rule['osm_key']
            if osm_key in feature:
                osm_value = str(feature[osm_key])
                if osm_value in rule['osm_values']:
                    gents.append({
                        'feature': feature,
                        'category': rule['gent_category'],
                        'label': rule['gent_label'],
                        'geomxy': geomxy,
                        'gtype': gtype,
                    })
print('%d features selected' % len(gents))



In [ ]:
for n in features['name'].values:
    if type(n) == str and 'pond' in n.lower():
        print(n)

In [ ]:
for f in features.to_dict('records'):
    if f['name'] == 'Rockrimmon Road':
        print(f['name'], f['highway'], f['geometry'].geom_type)

## Maps of the different types of features.

In [ ]:
pd.DataFrame(gents)[['category', 'gtype', 'label']].value_counts().sort_index()

In [ ]:
# Roads and waterways.
fig = make_subplots(rows=1, cols=1)
shown = set()
shown.add('land')

def add_features(label, color, linewidth):
    for gent in gents:
        if gent['label'] == label:
            draw_shape(gent['geomxy'], fig, irow=1, icol=1, 
                       name=label, color=color, linewidth=linewidth,
                       showlegend=(label not in shown))
            shown.add(label)

add_features('rivers and streams', color='blue', linewidth=2)
add_features('lakes and ponds', color='#71b8f0', linewidth=2)

add_features('land', color='#e0d8b0', linewidth=1)
add_features('highway', color='red', linewidth=3)
add_features('primary road', color='purple', linewidth=2)
add_features('secondary road', color='green', linewidth=2)
add_features('tertiary road', color='teal', linewidth=2)
add_features('residential road', color='gray', linewidth=1)

# Hack to mask out something.
for gent in gents:
    if gent['feature']['name'] == "Greenwood Pond":
        draw_shape(gent['geomxy'], fig, irow=1, icol=1, 
                   name="masked", color='black', linewidth=2,
                   showlegend=True)


lo = fig['layout']
lo['title'] = 'roadways'
lo['width'] = 600
lo['height'] = 540
lo['xaxis1']['range'] = [0, aoi_size]
lo['yaxis1']['range'] = [0, aoi_size]
lo['xaxis1']['showticklabels'] = False
lo['yaxis1']['showticklabels'] = False
fig


In [ ]:
# Roadways
fig = make_subplots(rows=1, cols=1)
shown = set()
shown.add('land')

def add_features(label, color, linewidth):
    for gent in gents:
        if gent['label'] == label:
            draw_shape(gent['geomxy'], fig, irow=1, icol=1, 
                       name=label, color=color, linewidth=linewidth,
                       showlegend=(label not in shown))
            shown.add(label)

add_features('land', color='#e0d8b0', linewidth=1)
add_features('highway', color='red', linewidth=3)
add_features('primary road', color='blue', linewidth=2)
add_features('secondary road', color='green', linewidth=2)
add_features('tertiary road', color='teal', linewidth=2)
add_features('residential road', color='gray', linewidth=1)

lo = fig['layout']
lo['title'] = 'roadways'
lo['width'] = 600
lo['height'] = 540
lo['xaxis1']['range'] = [0, aoi_size]
lo['yaxis1']['range'] = [0, aoi_size]
lo['xaxis1']['showticklabels'] = False
lo['yaxis1']['showticklabels'] = False
fig

In [ ]:
# Waterways
fig = make_subplots(rows=1, cols=1)
shown = set()
shown.add('land')
shown.add('highway')
shown.add('primary road')
shown.add('secondary road')
shown.add('tertiary teal')
shown.add('residential road')

add_features('land', color='#e0d8b0', linewidth=1)
add_features('highway', color='#bbbbbb', linewidth=1)
add_features('primary road', color='#bbbbbb', linewidth=1)
add_features('secondary road', color='#bbbbbb', linewidth=1)
add_features('tertiary teal', color='#bbbbbb', linewidth=1)
add_features('residential road', color='#bbbbbb', linewidth=1)

add_features('rivers and streams', color='blue', linewidth=2)
add_features('lakes and ponds', color='#71b8f0', linewidth=2)

lo = fig['layout']
lo['title'] = 'waterways'
lo['width'] = 600
lo['height'] = 540
lo['xaxis1']['range'] = [0, aoi_size]
lo['yaxis1']['range'] = [0, aoi_size]
lo['xaxis1']['showticklabels'] = False
lo['yaxis1']['showticklabels'] = False
fig

In [ ]:
# Amenities
fig = make_subplots(rows=1, cols=1)
shown = set()
shown.add('land')
shown.add('highway')
shown.add('primary road')
shown.add('secondary road')
shown.add('tertiary teal')
shown.add('residential road')

add_features('land', color='#e0d8b0', linewidth=1)
add_features('highway', color='#bbbbbb', linewidth=1)
add_features('primary road', color='#bbbbbb', linewidth=1)
add_features('secondary road', color='#bbbbbb', linewidth=1)
add_features('tertiary teal', color='#bbbbbb', linewidth=1)
add_features('residential road', color='#bbbbbb', linewidth=1)

add_features('parking lot', color='gray', linewidth=2)
add_features('commercial', color='blue', linewidth=2)
add_features('food and drink', color='purple', linewidth=2)
add_features('recreation', color='green', linewidth=2)

lo = fig['layout']
lo['title'] = 'amenities'
lo['width'] = 600
lo['height'] = 540
lo['xaxis1']['range'] = [0, aoi_size]
lo['yaxis1']['range'] = [0, aoi_size]
lo['xaxis1']['showticklabels'] = False
lo['yaxis1']['showticklabels'] = False
fig

In [ ]:
# landuse
fig = make_subplots(rows=1, cols=1)
shown = set()
shown.add('land')
shown.add('highway')
shown.add('primary road')
shown.add('secondary road')
shown.add('tertiary teal')
shown.add('residential road')

add_features('land', color='#e0d8b0', linewidth=1)
add_features('highway', color='#bbbbbb', linewidth=1)
add_features('primary road', color='#bbbbbb', linewidth=1)
add_features('secondary road', color='#bbbbbb', linewidth=1)
add_features('tertiary teal', color='#bbbbbb', linewidth=1)
add_features('residential road', color='#bbbbbb', linewidth=1)

add_features('commercial', color='blue', linewidth=2)
add_features('agricultural', color='green', linewidth=2)
add_features('forest', color='darkgreen', linewidth=2)
add_features('residential', color='magenta', linewidth=2)
add_features('industrial', color='darkgray', linewidth=2)
add_features('retail', color='orange', linewidth=2)

lo = fig['layout']
lo['title'] = 'landuse'
lo['width'] = 600
lo['height'] = 540
lo['xaxis1']['range'] = [0, aoi_size]
lo['yaxis1']['range'] = [0, aoi_size]
lo['xaxis1']['showticklabels'] = False
lo['yaxis1']['showticklabels'] = False
fig